<a href="https://colab.research.google.com/github/FacundoR26/groundwater-dependent-forests-sanjuan/blob/main/DEM_bosques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extraccion de variables topograficas a partir de un Modelo de elevacion digital**


In [ ]:
!pip install localtileserver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 16.7 MB/s eta 0:00:00


In [ ]:
import ee
import geemap
import pandas as pd
import datetime
import math
import os

Iniciaremos sesion con un proyecto de gee

In [ ]:
# Inicializar Earth Engine
try:
    ee.Initialize(project="identificacion-de-bosques")
except ee.EEException as e:
       ee.Authenticate()
       ee.Initialize(project="identificacion-de-bosques")

Usaremos un archivo shape importado de GEE assets

In [ ]:
# Replace 'users/your_username/your_asset_path' with the actual asset ID of your shapefile in Earth Engine
shapefile_asset_id = 'users/facu_ruarte/puntos_freatica_exp'

# Load the shapefile asset from Earth Engine
try:
    roi_feature_collection = ee.FeatureCollection(shapefile_asset_id)
    print(f"Successfully loaded shapefile asset: {shapefile_asset_id}")

    # Assign the loaded FeatureCollection to the user_roi property of the Map object
    # Check if Map object exists, if not, create one
    if 'Map' not in locals():
        Map = geemap.Map()
        print("geemap.Map() object created.")

    Map.user_roi = roi_feature_collection
    print("Assigned loaded shapefile to Map.user_roi.")

    # Center the map on the loaded shapefile
    Map.centerObject(roi_feature_collection)
    print("Centered map on the loaded shapefile.")

    # You can optionally add the shapefile to the map as a layer for visualization
    Map.addLayer(roi_feature_collection, {}, 'Region of Interest (from Asset)')
    print("Added shapefile layer to the map.")

    # Display the map
    display(Map)

except ee.EEException as e:
    print(f"Error loading Earth Engine asset: {e}")
    print(f"Please ensure the asset ID '{shapefile_asset_id}' is correct and the asset is publicly accessible or in your Earth Engine account.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully loaded shapefile asset: users/facu_ruarte/puntos_freatica_exp
geemap.Map() object created.
An unexpected error occurred: property 'user_roi' of 'Map' object has no setter


Para elegir un area diferente mediante la herramienta de geemap (En caso de querer cambiar el roi)

In [ ]:
# Seleccion de poligono de interes
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Para exportar el clip realizado con geemap a los assets de gee (Ejecutar en caso de hacer el clip virtual)

In [ ]:
# Export the clip to gee assets
if hasattr(Map, 'user_roi') and Map.user_roi is not None:
    # Convert the user_roi geometry to a FeatureCollection
    feature_collection = ee.FeatureCollection(Map.user_roi)

    # Define the Earth Engine asset ID where you want to save the shapefile
    earth_engine_asset_id = 'users/facu_ruarte/Sur_SanJuan_Export'

    # Export the FeatureCollection to an Earth Engine asset
    task = ee.batch.Export.table.toAsset(
        collection=feature_collection,
        description='Sur_SanJuan_Export',
        assetId=earth_engine_asset_id
    )

    task.start()

    print(f"Export task started for user_roi to asset ID: {earth_engine_asset_id}")
    print("Monitor the task progress in the Earth Engine Tasks tab.")
else:
    print("No user_roi found on the map to export.")

No user_roi found on the map to export.


# DEM
Crearemos un modelo de elevación digital utilizando los datos del sensor RSTM de Google Earth Engine para un polígono de interés.

## Seleccionar el sensor rstm
Identificaremos la colección de imágenes correspondiente al sensor RSTM en Google Earth Engine.


In [ ]:
# Buscar la colección de imágenes SRTM
srtm_collection_id = 'USGS/SRTMGL1_003'

# Cargar la colección de imágenes SRTM
srtm = ee.ImageCollection(srtm_collection_id)
print(f"SRTM collection ID: {srtm_collection_id}")

SRTM collection ID: USGS/SRTMGL1_003


## Filtrar la colección
Filtraremos la colección de imágenes por fecha y por la extensión geográfica del polígono de interés.


In [ ]:
# Define the SRTM image asset ID
srtm_image_id = 'USGS/SRTMGL1_003'

# Load the SRTM image
srtm_image = ee.Image(srtm_image_id)

# Filter the SRTM image by spatial extent using the loaded FeatureCollection
# Check if roi_feature_collection is defined and not None
if 'roi_feature_collection' in locals() and roi_feature_collection is not None:
  filtered_srtm_image = srtm_image.clip(roi_feature_collection)
  print("Filtered SRTM image by roi_feature_collection.")
else:
  print("Warning: roi_feature_collection not defined. Using the full SRTM image.")
  filtered_srtm_image = srtm_image

# Print information about the filtered image to verify
print("Filtered SRTM image information:", filtered_srtm_image.getInfo())

Filtered SRTM image by roi_feature_collection.
Filtered SRTM image information: {'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [1296001, 417601], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}], 'version': 1641990767055141, 'id': 'USGS/SRTMGL1_003', 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'Image', 'keywords': ['dem', 'elevation', 'geophysical', 'nasa', 'srtm', 'topography', 'usgs'], 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="https://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by N

## Visualizar el modelo de elevación
Mostraremos el modelo de elevación digital en el mapa.

In [ ]:
# Define visualization parameters for the elevation data
elevation_viz = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'FF8040', '000000']
}

# Add the filtered SRTM image to the map
Map.addLayer(filtered_srtm_image, elevation_viz, 'SRTM Elevation Model')

# Display the map (if not already displayed)
display(Map)

Map(bottom=19603.0, center=[-30.51494904517772, -66.27502441406251], controls=(WidgetControl(options=['positio…

# Extraccion de variables del DEM
Obtendremos el modelo de elevación digital (DEM) a partir de los datos del sensor SRTM de Google Earth Engine, calcula la pendiente, orientación, curvatura a partir de este DEM, y visualiza todas estas capas en el mapa.

## Calcular la pendiente (slope)
Calcularemos la pendiente a partir del modelo de elevación digital utilizando las funciones de Earth Engine.


In [ ]:
# Select the elevation band from the filtered SRTM image
elevation_band = filtered_srtm_image.select('elevation')

# Compute the slope from the elevation band
slope_image = ee.Terrain.slope(elevation_band)

# Print information about the slope image to verify
print("Slope image information:", slope_image.getInfo())

Slope image information: {'type': 'Image', 'bands': [{'id': 'slope', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 90}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}]}


## Calcular la orientación (aspect)
Calcularemos la orientación (dirección de la pendiente) a partir del modelo de elevación digital.


In [ ]:
# Select the elevation band from the filtered SRTM image (already done in the previous step, but doing it again for clarity)
elevation_band = filtered_srtm_image.select('elevation')

# Compute the aspect from the elevation band
aspect_image = ee.Terrain.aspect(elevation_band)

# Print information about the aspect image to verify
print("Aspect image information:", aspect_image.getInfo())

Aspect image information: {'type': 'Image', 'bands': [{'id': 'aspect', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 360}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}]}


## Exportar el modelo de elevación
Guardaremos el modelo de elevación digital como un archivo GeoTIFF en Drive.

In [ ]:
# Export the image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=filtered_srtm_image,
    description='SRTM_Elevation_Model_DEM',
    folder='earth_engine_exports', # Nombre de la carpeta
    fileNamePrefix='srtm_elevation',
    region=roi_feature_collection.geometry(), # Export the clipped region using the ROI geometry
    scale=30, # Set the export resolution (e.g., 30 meters for SRTMGL1_003)
    crs='EPSG:4326' # Set the coordinate reference system (mejor usar UTM?)
)

task.start()

print("Export task started. You can monitor its progress in the Earth Engine Tasks tab.")

Export task started. You can monitor its progress in the Earth Engine Tasks tab.


Exportaremos el aspect

In [ ]:
# Select the elevation band from the filtered SRTM image (already done in previous steps)
elevation_band = filtered_srtm_image.select('elevation')

# Compute the aspect from the elevation band
aspect_image = ee.Terrain.aspect(elevation_band)

# Configure the export task for the aspect_image
task = ee.batch.Export.image.toDrive(
    image=aspect_image,
    description='Aspect_Export',
    folder='earth_engine_exports',
    fileNamePrefix='aspect',
    region=aspect_image.geometry(),
    scale=30,
    crs='EPSG:4326'
)

# Start the export task
task.start()

# Print confirmation message
print("Aspect export task started. You can monitor its progress in the Earth Engine Tasks tab.")

Aspect export task started. You can monitor its progress in the Earth Engine Tasks tab.


Exportaremos la pendiente

In [ ]:
# Select the elevation band from the filtered SRTM image
elevation_band = filtered_srtm_image.select('elevation')

# Compute the slope from the elevation band
slope_image = ee.Terrain.slope(elevation_band)

# Export the slope_image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=slope_image,
    description='Slope_Export',
    folder='earth_engine_exports',
    fileNamePrefix='slope',
    region=slope_image.geometry(),
    scale=30,
    crs='EPSG:4326'
)

task.start()

print("Slope export task started. You can monitor its progress in the Earth Engine Tasks tab.")

Slope export task started. You can monitor its progress in the Earth Engine Tasks tab.


Una vez obtenida la acumulacion de flujo, tendremos los dos componentes principales necesarios para calcular el TWI a 30 metros:

1.  La capa de **pendiente** (`slope_image`), calculada previamente a partir del SRTM de 30m.
2.  La capa de **acumulación de flujo** (`flow_accumulation_ee_image`), cargada desde el asset (y que debe estar a 30m de resolución).

El siguiente paso será combinar estas dos capas utilizando la fórmula del TWI.

In [ ]:
flow_accumulation_asset_id = 'projects/identificacion-de-bosques/assets/Acumulacion_flujo' # <<< REPLACE WITH YOUR ASSET ID

# Load the flow accumulation image asset from Earth Engine
try:
    flow_accumulation_ee_image = ee.Image(flow_accumulation_asset_id)
    print(f"Successfully loaded Earth Engine asset: {flow_accumulation_asset_id}")

    # Print image info to verify bands and properties
    print("Flow Accumulation Earth Engine image information:", flow_accumulation_ee_image.getInfo())

    # Define visualization parameters for the flow accumulation image.
    # Adjust min/max based on the range of values in your specific flow accumulation data
    flow_accumulation_viz = {
      'min': 0,        # Minimum flow accumulation (single cell)
      'max': 2900000,    # Example max value for visualization (adjust based on your data)
      'palette': ['ffffff', '00ffff', '0080ff', '0000ff', '000080'], # White (low) to deep blue (high)
      # 'scale': 'log' # Consider using a logarithmic scale for visualization if values have a large range
    }

    # Add the flow accumulation layer to the Map
    Map.addLayer(flow_accumulation_ee_image, flow_accumulation_viz, 'Flow Accumulation (30m from SAGA)')

    print(f"Added '{flow_accumulation_asset_id}' layer to the map.")

    # Display the map (if not already displayed)
    display(Map)

except ee.EEException as e:
    print(f"Error loading Earth Engine asset: {e}")
    print(f"Please ensure the asset ID '{flow_accumulation_asset_id}' is correct and the asset is publicly accessible or in your Earth Engine account.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully loaded Earth Engine asset: projects/identificacion-de-bosques/assets/Acumulacion_flujo
Flow Accumulation Earth Engine image information: {'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [4658, 2640], 'crs': 'EPSG:4326', 'crs_transform': [0.0002694946, 0, -68.59364829629999, 0, -0.0002694946, -31.531674917400004]}], 'version': 1755817449533226, 'id': 'projects/identificacion-de-bosques/assets/Acumulacion_flujo', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-68.59351329607513, -31.531539717375257], [-68.59383024541242, -31.531748464734886], [-68.59383270167875, -32.243275649771995], [-68.43663400543447, -32.24327588504463], [-68.27975446180832, -32.2432759071549], [-68.12287494178827, -32.24327587552803], [-67.96599535349691, -32.243275849783295], [-67.80911587376808, -32.243275927359456], [-67.65223631175203, -32.24327588086157], [-67.49535674606437, -32.243275850059106], [-67.3381

Map(bottom=78105.0, center=[-31.907873056360206, -67.4523388070433], controls=(WidgetControl(options=['positio…

Observaremos el rango de valores de la acumulacion de flujo

In [ ]:
# Inspect the properties of the flow_accumulation_ee_image to find the data range
# We can use reduceRegion to get min and max values for a small area or getInfo() for general properties.

print("Inspecting flow_accumulation_ee_image to find the data range...")

# Attempt to compute min and max values over the image.
# This can be computationally intensive for large areas.
# We can use a reducer to get the min and max values within the region of interest.

if 'roi_feature_collection' in locals() and roi_feature_collection is not None:
    region_of_interest = roi_feature_collection.geometry()
else:
    print("Warning: roi_feature_collection not found. Cannot compute min/max over the specific ROI.")
    region_of_interest = flow_accumulation_ee_image.geometry() # Use the image geometry as a fallback


# Compute min and max values. This is an Earth Engine computation that will run when evaluated.
min_max_values = flow_accumulation_ee_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=region_of_interest,
    scale=30, # Use the scale of your flow accumulation data
    maxPixels=1e9 # Increase maxPixels if your region is large
)

# Get the results. This will trigger the Earth Engine computation.
min_value = min_max_values.get('b1_min').getInfo()
max_value = min_max_values.get('b1_max').getInfo()

print(f"Estimated minimum flow accumulation value: {min_value}")
print(f"Estimated maximum flow accumulation value: {max_value}")

# Now, update the visualization parameters with these estimated values.
flow_accumulation_viz_updated = {
  'min': min_value if min_value > 0 else 1e-7, # Use a small positive value if min is zero or negative for log scale
  'max': max_value,
  'palette': ['ffffff', '00ffff', '0080ff', '0000ff', '000080'], # White (low) to deep blue (high) palette
  'scale': 'log' # Use a logarithmic scale for visualization
}

# Remove the previous flow accumulation layer if it exists to avoid duplicates.
layers_to_remove = ['Flow Accumulation (30m from SAGA)', 'Flow Accumulation (Updated Viz)']
for layer_name in layers_to_remove:
    for layer in Map.layers:
        if hasattr(layer, 'name') and layer.name == layer_name:
            Map.remove_layer(layer)
            print(f"Removed existing '{layer_name}' layer.")
            break # Assuming there's only one such layer


# Add the flow accumulation layer again with the updated visualization parameters.
Map.addLayer(flow_accumulation_ee_image, flow_accumulation_viz_updated, 'Flow Accumulation (Log Scale Viz)')

print("Added 'Flow Accumulation (Log Scale Viz)' layer to the map with updated visualization.")

# Display the map to see the changes
display(Map)

Inspecting flow_accumulation_ee_image to find the data range...
Estimated minimum flow accumulation value: 1.452546740665639e-07
Estimated maximum flow accumulation value: 0.3591213524341583
Removed existing 'Flow Accumulation (30m from SAGA)' layer.
Added 'Flow Accumulation (Log Scale Viz)' layer to the map with updated visualization.


Map(bottom=78105.0, center=[-31.907873056360206, -67.4523388070433], controls=(WidgetControl(options=['positio…

## Visualizar los derivados del dem
Añadiremos las capas de pendiente, orientación, curvatura y acumulación de flujo al mapa para su visualización.


In [ ]:
# --- Calculation of TWI ---
# The Topographic Wetness Index (TWI) is calculated as: ln(Flow Accumulation / tan(Slope))
# We need the flow_accumulation_ee_image and the slope_image.

# Ensure slope is in radians for the tan function if the slope image is in degrees.
# The ee.Terrain.slope function outputs slope in degrees by default.
# We need to convert degrees to radians: radians = degrees * (pi / 180)
slope_radians = slope_image.divide(180).multiply(ee.Number(math.pi))

# Avoid division by zero or taking the logarithm of zero or negative values.
# Replace zero or near-zero slope values with a small positive number before taking the tangent.
# Replace zero or near-zero flow accumulation values with a small positive number before taking the logarithm.
# A common practice for slope is to add a small value or replace zeros to avoid issues with tan(0).
# Let's replace very small or zero slopes with a minimum slope value to avoid tan(0) = 0 and subsequent division by zero.
min_slope = 0.001 # Define a small minimum slope value (in degrees before conversion to radians)
slope_corrected = slope_image.max(min_slope) # Replace values less than min_slope with min_slope
slope_radians_corrected = slope_corrected.divide(180).multiply(ee.Number(math.pi))

# Calculate the tangent of the slope in radians
tan_slope = slope_radians_corrected.tan()

# Replace zero or near-zero flow accumulation values with a small positive number before taking the logarithm.
min_flow_acc = 1 # Define a small minimum flow accumulation value (a single cell)
flow_accumulation_corrected = flow_accumulation_ee_image.max(min_flow_acc)

# Calculate TWI: ln(Flow Accumulation / tan(Slope))
twi_image = flow_accumulation_corrected.divide(tan_slope).log()

# Rename the band for clarity
twi_image = twi_image.rename('TWI')


# --- Visualization of TWI ---

# Define visualization parameters for the TWI image.

# Compute min/max of the TWI image for better visualization.
# This can be computationally intensive. We'll use the roi_feature_collection geometry if available.
if 'roi_feature_collection' in locals() and roi_feature_collection is not None:
    twi_region = roi_feature_collection.geometry()
else:
    # Fallback to the TWI image geometry if ROI is not available
    twi_region = twi_image.geometry()
    print("Warning: roi_feature_collection not found. Computing TWI min/max over TWI image geometry.")

print("Computing min/max for TWI image...")
twi_min_max = twi_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=twi_region,
    scale=30, # Use the appropriate scale
    maxPixels=1e9
)

# Get the results. This will trigger the Earth Engine computation.
# Check if the keys exist in the dictionary before getting them
twi_min = twi_min_max.get('TWI_min').getInfo() if twi_min_max.get('TWI_min') is not None else None
twi_max = twi_min_max.get('TWI_max').getInfo() if twi_min_max.get('TWI_max') is not None else None

if twi_min is not None and twi_max is not None:
    print(f"Estimated minimum TWI value: {twi_min}")
    print(f"Estimated maximum TWI value: {twi_max}")

    # Define TWI visualization parameters based on computed range
    # Using a palette suitable for wetness index (e.g., blues for high TWI, browns for low TWI)
    twi_viz = {
      'min': twi_min,
      'max': twi_max,
      'palette': ['brown', 'yellow', 'green', 'cyan', 'blue'] # Example palette
    }

    # Add the TWI layer to the Map
    Map.addLayer(twi_image, twi_viz, 'Topographic Wetness Index (TWI)')

    print("Calculated and added TWI layer to the map.")
else:
    print("Could not compute min/max for TWI image. Skipping visualization.")


# Display the map to visualize the TWI layer
display(Map)

Computing min/max for TWI image...
Estimated minimum TWI value: -1.2351169855529114
Estimated maximum TWI value: 10.955982243921408
Calculated and added TWI layer to the map.


Map(bottom=78105.0, center=[-31.907873056360206, -67.4523388070433], controls=(WidgetControl(options=['positio…

## Exportar el DEM a Earth Engine Assets

Exportaremos el modelo de elevación digital (`filtered_srtm_image`) a un asset en Google Earth Engine.

In [ ]:
# Export the filtered SRTM image to an Earth Engine asset
# Use the roi_feature_collection geometry for export region to ensure it's bounded
if 'roi_feature_collection' in locals() and roi_feature_collection is not None:
    export_region = roi_feature_collection.geometry()
else:
    # Fallback to the image geometry, but print a warning as this might cause issues if not properly clipped
    export_region = filtered_srtm_image.geometry()
    print("Warning: roi_feature_collection not found. Using filtered_srtm_image geometry for export, which might be unbounded.")


dem_export_task = ee.batch.Export.image.toAsset(
    image=filtered_srtm_image,
    description='SRTM_DEM_Asset_Export',
    assetId='users/facu_ruarte/SRTM_DEM_clipped',
    region=export_region, # Use the defined export_region
    scale=30, # Ensure the scale matches the original data
    crs='EPSG:32719' # Use an appropriate CRS
)

# Start the export task
dem_export_task.start()

print("DEM export task to asset started. Monitor its progress in the Earth Engine Tasks tab.")

DEM export task to asset started. Monitor its progress in the Earth Engine Tasks tab.


## Exportar la Pendiente a Earth Engine Assets

Exportaremos la capa de pendiente (`slope_image`) a un asset en Google Earth Engine.

In [ ]:
# Export the slope image to an Earth Engine asset
# Use the roi_feature_collection geometry for export region to ensure it's bounded
if 'roi_feature_collection' in locals() and roi_feature_collection is not None:
    export_region = roi_feature_collection.geometry()
else:
    # Fallback to the image geometry, but print a warning
    export_region = slope_image.geometry()
    print("Warning: roi_feature_collection not found. Using slope_image geometry for export, which might be unbounded.")

slope_export_task = ee.batch.Export.image.toAsset(
    image=slope_image,
    description='Slope_Asset_Export',
    assetId='users/facu_ruarte/Slope_clipped',
    region=export_region, # Use the defined export_region
    scale=30, # Ensure the scale matches the original data
    crs='EPSG:32719' # Use an appropriate CRS
)

# Start the export task
slope_export_task.start()

print("Slope export task to asset started. Monitor its progress in the Earth Engine Tasks tab.")

Slope export task to asset started. Monitor its progress in the Earth Engine Tasks tab.


## Exportar el TWI a Earth Engine Assets

Exportaremos la capa del Índice de Humedad Topográfica (TWI) (`twi_image`) a un asset en Google Earth Engine.

In [ ]:
# Export the TWI image to an Earth Engine asset
# Use the roi_feature_collection geometry for export region to ensure it's bounded
if 'roi_feature_collection' in locals() and roi_feature_collection is not None:
    export_region = roi_feature_collection.geometry()
else:
    # Fallback to the image geometry, but print a warning
    export_region = twi_image.geometry()
    print("Warning: roi_feature_collection not found. Using twi_image geometry for export, which might be unbounded.")

twi_export_task = ee.batch.Export.image.toAsset(
    image=twi_image,
    description='TWI_Asset_Export',
    assetId='users/facu_ruarte/TWI_clipped',
    region=export_region,
    scale=30,
    crs='EPSG:32719'
)

# Start the export task
twi_export_task.start()

print("TWI export task to asset started. Monitor its progress in the Earth Engine Tasks tab.")

TWI export task to asset started. Monitor its progress in the Earth Engine Tasks tab.
